# 從零訓練 Transformer (Training Transformer from Scratch)

本 notebook 對應李宏毅老師 2025 Spring ML HW4，實作從零訓練一個小型 Transformer。

## 學習目標
1. 實作完整的 Transformer 架構
2. 理解 Tokenizer 的設計
3. 學習語言模型的訓練流程
4. 掌握訓練技巧（學習率調度、梯度累積等）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Tokenizer 實作

In [ ]:
class CharTokenizer:
    """簡單的字元級 Tokenizer"""
    def __init__(self):
        self.char_to_id = {}
        self.id_to_char = {}
        self.vocab_size = 0
        
        # 特殊 tokens
        self.pad_token = "<PAD>"
        self.unk_token = "<UNK>"
        self.bos_token = "<BOS>"
        self.eos_token = "<EOS>"
    
    def fit(self, texts):
        """從文本建立詞彙表"""
        chars = set()
        for text in texts:
            chars.update(text)
        
        # 加入特殊 tokens
        all_tokens = [self.pad_token, self.unk_token, self.bos_token, self.eos_token] + sorted(chars)
        
        self.char_to_id = {c: i for i, c in enumerate(all_tokens)}
        self.id_to_char = {i: c for c, i in self.char_to_id.items()}
        self.vocab_size = len(all_tokens)
        
        return self
    
    def encode(self, text):
        return [self.char_to_id.get(c, self.char_to_id[self.unk_token]) for c in text]
    
    def decode(self, ids):
        return ''.join([self.id_to_char.get(i, self.unk_token) for i in ids])

# 測試
tokenizer = CharTokenizer()
tokenizer.fit(["Hello world!", "How are you?"])
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Encoded: {tokenizer.encode('Hello')}")
print(f"Decoded: {tokenizer.decode([8, 13, 16, 16, 17])}")

## 2. 完整 Transformer 架構

In [ ]:
class TransformerBlock(nn.Module):
    """單個 Transformer Block"""
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Self-attention with residual
        attn_out, _ = self.attention(x, x, x, attn_mask=mask, is_causal=True)
        x = self.norm1(x + self.dropout(attn_out))
        # FFN with residual
        x = self.norm2(x + self.ff(x))
        return x


class GPTModel(nn.Module):
    """簡化版 GPT 模型"""
    def __init__(self, vocab_size, d_model=256, num_heads=4, num_layers=4, d_ff=512, max_len=512, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_len, d_model)
        
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        # 權重綁定
        self.lm_head.weight = self.token_embedding.weight
        
        self._init_weights()
    
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, std=0.02)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, std=0.02)
    
    def forward(self, x):
        B, T = x.shape
        
        # Embeddings
        tok_emb = self.token_embedding(x)
        pos_emb = self.position_embedding(torch.arange(T, device=x.device))
        x = tok_emb + pos_emb
        
        # Transformer blocks
        for block in self.blocks:
            x = block(x)
        
        x = self.norm(x)
        logits = self.lm_head(x)
        
        return logits
    
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """自回歸生成"""
        for _ in range(max_new_tokens):
            logits = self(idx[:, -512:])  # 截斷到 max_len
            logits = logits[:, -1, :] / temperature
            
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = float('-inf')
            
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_token], dim=1)
        
        return idx

# 測試模型
model = GPTModel(vocab_size=100, d_model=128, num_heads=4, num_layers=2)
x = torch.randint(0, 100, (2, 32))
logits = model(x)
print(f"Model output shape: {logits.shape}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## 3. 訓練流程

In [ ]:
class TextDataset(Dataset):
    """語言模型資料集"""
    def __init__(self, text, tokenizer, block_size=128):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
    
    def __len__(self):
        return max(0, len(self.data) - self.block_size - 1)
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y


def train_model(model, train_loader, epochs=5, lr=3e-4):
    """訓練函數"""
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            
            logits = model(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        scheduler.step()
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return losses

# 範例訓練
sample_text = "The quick brown fox jumps over the lazy dog. " * 100
tokenizer = CharTokenizer().fit([sample_text])
dataset = TextDataset(sample_text, tokenizer, block_size=64)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = GPTModel(vocab_size=tokenizer.vocab_size, d_model=64, num_heads=2, num_layers=2)
print("\n開始訓練...")
losses = train_model(model, loader, epochs=3)

## 4. 總結

### 訓練 Transformer 的關鍵技巧
1. **權重初始化**：使用小的標準差（如 0.02）
2. **學習率調度**：Warmup + Cosine Decay
3. **梯度裁剪**：防止梯度爆炸
4. **權重綁定**：輸入輸出 embedding 共享
5. **混合精度訓練**：加速訓練、節省記憶體

In [ ]:
print("從零訓練 Transformer - 完成！")